In [1]:
import numpy as np
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline
import spacy

In [2]:
from JOSS_PDF_Cleaner import Clean_PDF
import re
from termcolor import colored
import warnings

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df_reviewers = pd.read_csv('../Data/JOSS Table Test.csv')

In [5]:
def GetReviewer_Vectors(df_reviewers=df_reviewers):
    reviewer_vectors = np.zeros(((df_reviewers.shape[0]-1),300))
    for i in range(df_reviewers.shape[0]-1):
        #if i%10 == 0:
        #    print(i)
        if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i]) == False:
            review_text = df_reviewers['Domains/topic areas you are comfortable reviewing'].iloc[1:].values[i].lower()
            review_text = review_text.replace('-\\n','')
            review_text = review_text.replace('\\n',' ')
            review_text = review_text.replace('\n', ' ')
        
            review_arr = []
            for token in model(review_text):
                if token.is_alpha == True:
                    if token.is_stop == False:
                        review_arr.append(str(token.lemma_).lower())
            review_arr = np.array(review_arr)
        
            review_str = ''
            for j in np.unique(review_arr):
                review_str = review_str + j +' '
        
        #print(model(review_str).vector.shape)
        #print(reviewer_vectors.shape)
            reviewer_vectors[i] = model(review_str).vector
    
    return reviewer_vectors

In [6]:
def GetCosineSims(doc_vec, review_vec, df_reviewers=df_reviewers):
    all_usernames = []
    all_domains = []
    all_cosine_sims = []
    for j in range(len(review_vec)):
        if pd.isna(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing']) == False:
            all_cosine_sims.append(cosine_similarity(np.array([doc_vec]), np.array([review_vec[j]]))[0,0])
            all_domains.append(df_reviewers.iloc[j+1]['Domains/topic areas you are comfortable reviewing'].lower())
            all_usernames.append(df_reviewers.iloc[j+1].username)
    all_usernames= np.array(all_usernames)
    all_domains= np.array(all_domains)
    all_cosine_sims= np.array(all_cosine_sims)
    
    return all_usernames, all_domains, all_cosine_sims

In [7]:
PAPER_OF_INTEREST_FNAME  = glob.glob('/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/*/*/*.pdf')
K = 260
print(PAPER_OF_INTEREST_FNAME[K])
Paper_interest = PAPER_OF_INTEREST_FNAME[K]

/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.00592/10.21105.joss.00592.pdf


In [8]:
def MakeGreenRedText(Paper_interest, printer=True):
    texts = ''
    arr = []
    from pdfminer.high_level import extract_pages
    from pdfminer.layout import LTTextContainer
    for page_layout in extract_pages(Paper_interest):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                score = Clean_PDF(element.get_text().lower())
            #print(score)
                if score == 0:
                    if printer == True:
                        print(colored(element.get_text().lower(), 'green'))
                    arr.append(element.get_text())
                    texts = texts  + element.get_text() + ' '
                else:
                    if printer == True:
                        print(colored(element.get_text().lower(), 'red'))
            #arr.append(element.get_text())
            #texts = texts  + element.get_text() + ' '
    arr = np.array(arr)
    return texts, arr

In [9]:
texts, arr = MakeGreenRedText(Paper_interest)

summary

doi: 10.21105/joss.00592

software

• review
• repository
• archive

submitted: 12 december 2017
published: 15 june 2018

license
authors of papers retain copyright
and release the work under a cre-
ative commons attribution 4.0 in-
ternational license (cc-by).

aronnax: an idealised isopycnal ocean model

edward w. doddridge1 and alexey radul2

1 earth, atmospheric and planetary science, massachusetts institute of technology, cambridge,
ma, usa 2 brain and cognitive science, massachusetts institute of technology, cambridge, ma,
usa

aronnax is a highly idealised model for simulating large-scale (100-1000km) flows in the
ocean. aronnax is intended for theoretical and empirical oceanographers, as a (relatively)
fast and easy-to-use simulation model, bridging the gap between pencil-and-paper on one
hand, and more faithful (and complex) computational models on the other. the numerical
core is written in fortran to improve performance, and wrapped in python to improve
usability.



In [10]:
texts = texts.replace('-\\n','')
texts = texts.replace('-\n','')
texts = texts.replace('\\n',' ')
texts = texts.replace('\n', ' ')

In [11]:
texts

'Aronnax: An idealised isopycnal ocean model  Aronnax is a highly idealised model for simulating large-scale (100-1000km) flows in the ocean. Aronnax is intended for theoretical and empirical oceanographers, as a (relatively) fast and easy-to-use simulation model, bridging the gap between pencil-and-paper on one hand, and more faithful (and complex) computational models on the other. The numerical core is written in Fortran to improve performance, and wrapped in Python to improve usability.  Aronnax is an isopyncal model: it approximates the ocean as a number of discrete homogeneous layers with spatially variable thicknesses. These layers are stacked vertically and the density difference between neighbouring layers is specified by the user. Other widely used vertical coordinates require solving the equations of motion at specified vertical levels where the density is allowed to vary (S. M. Griffies et al. 2000). Representing the large-scale ocean circulation in layers contributes to Ar

In [19]:
str(arr[0])

'Aronnax: An idealised isopycnal ocean model\n'

In [14]:
model = spacy.load('en_core_web_lg')

In [32]:
reviewer_vectors = GetReviewer_Vectors()

In [25]:
len(arr[0].split())

6

In [56]:
def Get_Paper_Vector_Idea6_SpaCy(arr,model):
    sum_vector_text = np.zeros((300))
    count = 0
    for J in range(len(arr)):
        sum_line_vector_text = model(str(arr[J])).vector
        counter = len(arr[J].split())
        
        average_line_vector_text = sum_line_vector_text/ counter
        sum_vector_text = sum_vector_text + average_line_vector_text
        count = count + 1
        
    average_vector_text = sum_vector_text / count
    return average_vector_text

In [57]:
average_vector_text = Get_Paper_Vector_Idea6_SpaCy(arr,model)

In [58]:
average_vector_text

array([ 2.67286105e-03,  1.54858098e-02, -2.79282204e-03,  1.42601641e-04,
       -2.91437718e-03,  1.10176613e-02,  1.04418883e-02, -4.73451464e-03,
       -6.38419164e-03,  6.40118897e-02, -1.31636982e-02,  7.15007884e-03,
        2.34565655e-02, -2.24208581e-03, -3.89240887e-03,  1.83498926e-02,
        3.68017106e-03,  7.64348204e-02, -1.32752517e-02, -4.71564785e-03,
        6.28664857e-03,  7.80864044e-03, -6.75044791e-03, -1.10325287e-03,
        9.44254885e-03, -1.32788948e-03, -1.94291722e-02, -4.60908945e-03,
        1.17336080e-02,  2.19284839e-02, -1.25826442e-02, -2.03626678e-03,
       -2.29459177e-03,  9.24605106e-04,  5.65669409e-03, -7.62882633e-03,
        2.15269558e-02,  2.14300754e-02,  4.81508979e-03,  5.26789954e-05,
        6.79326617e-03, -3.81034196e-03,  6.73827692e-04, -8.37322395e-03,
       -1.21220475e-02,  2.15867908e-03, -2.45128857e-03,  4.08916669e-03,
        1.04492959e-03, -1.51285969e-03, -2.53387860e-03,  1.45647728e-03,
        1.02658151e-02, -

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
all_usernames, all_domains, all_cosine_sims = GetCosineSims(average_vector_text, reviewer_vectors)

In [60]:
def TopReviewers(number=5, all_usernames=all_usernames, all_domains=all_domains, all_cosine_sims=all_cosine_sims):
    message = 'Hello.\n I have found ' +str(number) + ' possible reviewers for this paper.'+ '\n\n'
    for J in range(number):
        index = np.argsort(all_cosine_sims)[-1-J]
        #print(index)
        ps = 'I believe '+ colored(str(all_usernames[index]), 'green') + ' will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of ' + colored(str(all_cosine_sims[index])[:6], 'blue') + '. This reviewers domain interests are ' + colored(str(all_domains[index].replace('\n', ',')), 'red')
        message = message + ps + '\n\n'
    print(message)    

In [61]:
TopReviewers()

Hello.
 I have found 5 possible reviewers for this paper.

I believe rchew will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.6283. This reviewers domain interests are r/stats, social sciences

I believe sptennak will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.6237. This reviewers domain interests are data science, machine learning, deep learning, physics, materials science, signal processing, mineral physics

I believe josiahparry will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.6196. This reviewers domain interests are web scraping, apis, sociology, political science, social science, urban studies, open data, data packages, general purpose r work

I believe RDastgh1 will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.6079. This 

In [62]:
def Idea6_lemma_Text(arr):
    text_list = []
    for J in range(len(arr)):
        doc = model(str(arr[J]))
        
        doc_arr = []
        for token in doc:
            if token.is_alpha == True:
                if token.is_stop == False:
                    doc_arr.append(str(token.lemma_).lower())
        doc_arr = np.array(doc_arr)
        
        doc_arr_mod = ''
        for i in doc_arr:
            doc_arr_mod = doc_arr_mod + i +' '
        #print(doc_arr_mod)
        text_list.append(doc_arr_mod)
    return text_list

In [63]:
text_list_lemma = Idea6_lemma_Text(arr)

In [64]:
average_vector_text2 = Get_Paper_Vector_Idea6_SpaCy(text_list_lemma,model)

In [65]:
all_usernames2, all_domains2, all_cosine_sims2 = GetCosineSims(average_vector_text2, reviewer_vectors)

In [66]:
TopReviewers(5, all_usernames2, all_domains2, all_cosine_sims2)

Hello.
 I have found 5 possible reviewers for this paper.

I believe nicoguaro will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5560. This reviewers domain interests are physics, math

I believe isabel-campos-plasencia will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5446. This reviewers domain interests are physics, biology, mathematics

I believe jmborr will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5331. This reviewers domain interests are neutron science, biophysics, physical chemistry, physics

I believe vahtras will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5285. This reviewers domain interests are chemistry, physics

I believe kpeeters will be a good reviewer for this paper. Their domain interests and this paper hav

In [67]:
def Idea6_unique_lemma_Text(arr):
    text_list = []
    for J in range(len(arr)):
        doc = model(str(arr[J]))
        
        doc_arr = []
        for token in doc:
            if token.is_alpha == True:
                if token.is_stop == False:
                    doc_arr.append(str(token.lemma_).lower())
        doc_arr = np.array(doc_arr)
        doc_str = ''
        for i in np.unique(doc_arr):
            doc_str = doc_str + i +' '
        text_list.append(doc_str)
    return text_list

In [68]:
text_list_unique_lemma = Idea6_unique_lemma_Text(arr)

In [69]:
average_vector_text3 = Get_Paper_Vector_Idea6_SpaCy(text_list_unique_lemma,model)

In [70]:
all_usernames3, all_domains3, all_cosine_sims3 = GetCosineSims(average_vector_text3, reviewer_vectors)

In [71]:
TopReviewers(5,all_usernames3, all_domains3, all_cosine_sims3)

Hello.
 I have found 5 possible reviewers for this paper.

I believe nicoguaro will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5613. This reviewers domain interests are physics, math

I believe isabel-campos-plasencia will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5482. This reviewers domain interests are physics, biology, mathematics

I believe jmborr will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5414. This reviewers domain interests are neutron science, biophysics, physical chemistry, physics

I believe kpeeters will be a good reviewer for this paper. Their domain interests and this paper have a cosine similairity score of 0.5338. This reviewers domain interests are physics, maths, computer algebra

I believe vahtras will be a good reviewer for this paper. Their domain interests and 